In [1]:
import numpy as np
import matplotlib.pyplot as plt

import jlinops

In [10]:
jlinops.build_1d_first_order_derivative(3)

<3x3 MatrixLinearOperator with dtype=float64>

In [11]:
A = jlinops.build_2d_first_order_derivative((3,4))

In [9]:
jlinops.build_2d_first_order_derivative_split((3,4))

(<12x12 MatrixLinearOperator with dtype=float64>,
 <12x12 MatrixLinearOperator with dtype=float64>)

In [6]:
A + A

<24x12 _SumLinearOperator with dtype=float64>

# Matrix operator

In [88]:
from scipy.sparse.linalg._interface import MatrixLinearOperator
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import issparse

In [140]:
class MatrixOperator(MatrixLinearOperator):

    def __init__(self, A):

        super().__init__(A)


    def __matmul__(self, B):
        "Overload matrix multiplication symbol."

        if isinstance(B, MatrixOperator):
            # If hitting with a linear operator.
            return MatrixOperator( self.A.dot(B.A) )
        elif isinstance(B, LinearOperator):
            # If hitting with a LinearOperator, the best we can do is return a LinearOperator
            return self @ B
        elif isinstance(B, np.ndarray):
            # If hitting with a NumPy array, we can dot then return MatrixOperator.
            return MatrixOperator(self.A.dot(B))
        elif issparse(B):
            # If sparse, we can dot then return MatrixOperator.
            return MatrixOperator(self.dot(B))
        else:
            raise Exception("matmul not implemented for these types.")
        


    def __rmatmul__(self, B):
        if isinstance(B, MatrixOperator):
            # If hitting with a linear operator.
            return MatrixOperator( B.A.dot(self.A) )
        elif isinstance(B, LinearOperator):
            # If hitting with a LinearOperator, the best we can do is return a LinearOperator
            return B @ self
        elif isinstance(B, np.ndarray):
            # If hitting with a NumPy array, we can dot then return MatrixOperator.
            return MatrixOperator(B.dot(self.A))
        elif issparse(B):
            # If sparse, we can dot then return MatrixOperator.
            return MatrixOperator(B.dot(self.A))
        else:
            raise Exception("matmul not implemented for these types.")


    
    def _transpose(self):
        return MatrixOperator(self.A.T)
        
    T = property(_transpose)

    def _adjoint(self):
        return MatrixOperator(self.A.H)
    
    H = property(_adjoint)



    def __add__(self, B):
        """Overload matrix addition."""

        return MatrixOperator( self.A + B.A )
    
    def __sub__(self, B):
        """Overload matrix addition."""

        return self.__add__(-B)
    
    def __neg__(self):
        """Overload negation."""

        return MatrixOperator(-self.A)
    
    def __mul__(self, c):
        """Multiplication by a scalar."""
        return MatrixOperator(c*self.A)
    
    def __rmul__(self, c):
        """Multiplication by a scalar."""
        return MatrixOperator(c*self.A)




In [141]:
A = jlinops.build_1d_first_order_derivative(5, boundary="zero").A
A = MatrixOperator(A)

In [142]:
assert type( A @ A ) is MatrixOperator
assert type( A + A ) is MatrixOperator
assert type( -A ) is MatrixOperator
assert type( A - A ) is MatrixOperator
assert type( 3*A ) is MatrixOperator
assert type( A @ np.ones(A.shape).T ) is MatrixOperator

In [150]:
A.H.A.toarray()

array([[ 1., -1.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0.],
       [ 0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  1.]])